In [1]:
import cudf.pandas
cudf.pandas.install()

import pandas as pd

In [2]:
import cupy as cp

gpus = cp.cuda.runtime.getDeviceCount()
if gpus:
    print("Available GPUs:")
    for i in range(gpus):
        print(f"GPU {i}: {cp.cuda.runtime.getDeviceProperties(i)['name']}")
else:
    print("No GPU available.")


Available GPUs:
GPU 0: b'NVIDIA GeForce RTX 3060'


In [3]:
import os
import language_tool_python
from spellchecker import SpellChecker
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import spacy
from textblob import TextBlob
import dask.dataframe as dd
from dask.distributed import Client, progress
from dask import delayed, compute
from spellchecker import SpellChecker
import spacy
from textblob import TextBlob
import language_tool_python
import dask.dataframe as dd

In [4]:
import os
import gdown

def download_file_if_not_exists(file_url, output_path):
    """Скачивает файл с Google Drive, если он ещё не существует в указанной директории."""
    # Проверка наличия файла
    if os.path.exists(output_path):
        print(f"Файл '{output_path}' уже существует.")
    else:
        print(f"Файл '{output_path}' не найден. Начинаю загрузку...")
        gdown.download(file_url, output_path, quiet=False)
        print(f"Файл '{output_path}' успешно загружен.")

# Указываем URL и путь к файлу
file_url = 'https://drive.google.com/uc?id=15pofNbomaoUap41Rcn1uNGeiJIqFd2qe'
output_file_name = 'wildberries_reviews.csv.gz'  # Укажите реальное имя файла, которое хотите сохранить
output_path = os.path.join(os.getcwd(), output_file_name)  # Полный путь к файлу

download_file_if_not_exists(file_url, output_path)


Файл '/workspace/wildberries_reviews.csv.gz' не найден. Начинаю загрузку...


Downloading...
From (original): https://drive.google.com/uc?id=15pofNbomaoUap41Rcn1uNGeiJIqFd2qe
From (redirected): https://drive.google.com/uc?id=15pofNbomaoUap41Rcn1uNGeiJIqFd2qe&confirm=t&uuid=a3d7238a-5c03-4494-8f6c-3b9884f464e3
To: /workspace/wildberries_reviews.csv.gz
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 164M/164M [00:03<00:00, 52.5MB/s]

Файл '/workspace/wildberries_reviews.csv.gz' успешно загружен.


In [5]:
import subprocess

def install_spacy_model_if_not_exists(model_name):
    """Устанавливает модель spaCy, если она ещё не установлена."""
    try:
        # Получаем список установленных моделей
        result = subprocess.run(
            ['python', '-m', 'spacy', 'info'],
            capture_output=True, text=True, check=True
        )
        
        # Проверяем наличие модели в списке установленных
        if model_name in result.stdout:
            print(f"Модель {model_name} уже установлена.")
        else:
            print(f"Модель {model_name} не найдена. Устанавливаю...")
            subprocess.run(['python', '-m', 'spacy', 'download', model_name], check=True)
            print(f"Модель {model_name} успешно установлена.")
    
    except subprocess.CalledProcessError as e:
        print(f"Ошибка при выполнении команды: {e}")

# Замените 'ru_core_news_lg' на нужное имя модели
install_spacy_model_if_not_exists('ru_core_news_lg')

Модель ru_core_news_lg уже установлена.


In [6]:
import subprocess
from language_tool_python import LanguageTool
# Запуск команды
process = subprocess.Popen(
    ['find', '/', '-name', 'languagetool-server.jar'],
    stdout=subprocess.PIPE,
    stderr=subprocess.DEVNULL,
    text=True
)

# Чтение вывода
output, _ = process.communicate()

# # Удаление пробелов и пустых строк
# LANGUAGETOOL_DIR = [line.strip() for line in output.splitlines() if line.strip()][0]

# Функция для установки необходимых пакетов и загрузки LanguageTool, если еще не загружено
def setup_languagetool():
    if not os.path.isdir(output):
        print("Установка необходимых пакетов и загрузка LanguageTool...")
        os.system('apt update && apt install -y default-jre wget unzip')
        os.system('wget https://languagetool.org/download/LanguageTool-stable.zip && unzip -o LanguageTool-stable.zip')
    else:
        print("LanguageTool уже установлен.")
        
temp = LanguageTool('ru-RU', config={'maxSpellingSuggestions': 1})
temp.close()

Unzipping /tmp/tmpazc9gve0.zip to /root/.cache/language_tool_python.
Downloaded https://www.languagetool.org/download/LanguageTool-6.4.zip to /root/.cache/language_tool_python.


In [9]:
import threading
import queue
from tqdm import tqdm
import pandas as pd

# Параметры
NUM_TOOLS = 6  # Количество экземпляров LanguageTool
MAX_CHECK_THREADS = 2  # Количество потоков для каждого экземпляра LanguageTool

# Функция для создания экземпляра LanguageTool
def create_tool(tools_queue, pbar):
    tool = LanguageTool('ru-RU', config={'maxSpellingSuggestions': 1, 'maxCheckThreads': MAX_CHECK_THREADS})
    tools_queue.put(tool)
    pbar.update(1)

# Функция для исправления текста с использованием LanguageTool
def correct_text(text, tool):
    if not isinstance(text, str):
        return text
    try:
        corrected_text = tool.correct(text)
        return corrected_text if corrected_text != text else text
    except Exception as e:
        print(f"Ошибка исправления текста: {e}")
        return text

# Функция для обработки текста в потоке
def process_text(text_queue, results, progress_bar):
    while not text_queue.empty():
        idx, text, tool = text_queue.get()
        corrected_text = correct_text(text, tool)
        results.append((idx, corrected_text))
        text_queue.task_done()
        progress_bar.update(1)

# Функция для обработки данных
def process_data(file_path, num_rows, num_tools):
    # Создание экземпляров LanguageTool с использованием многопоточности и tqdm
    tools_queue = queue.Queue()
    threads = []
    with tqdm(total=num_tools, desc="Создание экземпляров LanguageTool") as pbar:
        for _ in range(num_tools):
            thread = threading.Thread(target=create_tool, args=(tools_queue, pbar))
            thread.start()
            threads.append(thread)

        for thread in threads:
            thread.join()

    tools = [tools_queue.get() for _ in range(num_tools)]

    # Чтение данных с использованием pandas
    df = pd.read_csv(file_path, compression='gzip', nrows=num_rows)

    # Инициализация очереди
    text_queue = queue.Queue()
    results = []

    # Заполнение очереди строками для обработки
    with tqdm(total=len(df), desc="Заполнение очереди строками") as pbar:
        for idx, row in df.iterrows():
            tool = tools[idx % len(tools)]
            text_queue.put((idx, row['review_full_text'], tool))
            pbar.update(1)

    # Инициализация прогресс-бара
    with tqdm(total=text_queue.qsize(), desc="Обработка текста") as progress_bar:
        # Запуск потоков для обработки текста
        threads = []
        for _ in range(num_tools):
            thread = threading.Thread(target=process_text, args=(text_queue, results, progress_bar))
            thread.start()
            threads.append(thread)

        # Ожидание завершения всех потоков
        text_queue.join()
        for thread in threads:
            thread.join()

    # Обновление DataFrame с исправленным текстом
    corrected_texts = [text for _, text in sorted(results, key=lambda x: x[0])]
    df['corrected_texts'] = corrected_texts

    # Сохранение обновленного DataFrame обратно в исходный файл
    df.to_csv("corrected_" + file_path, compression='gzip', index=False)
    print(f"Обработка завершена для {num_rows} строк и {num_tools} экземпляров LanguageTool.")
    print(f"Результаты сохранены в corrected_{file_path}")

# Запуск обработки для разного количества экземпляров LanguageTool и строк
test_cases = [(1000, NUM_TOOLS)]
for num_rows, num_tools in test_cases:
    process_data('wildberries_reviews.csv.gz', num_rows=num_rows, num_tools=num_tools)


Создание экземпляров LanguageTool:   0%|                                                                                                                                          | 0/6 [00:12<?, ?it/s]


MaybeEncodingError: Error sending result: '[LanguageTool(language=<LanguageTag "ru-RU">, motherTongue=None)]'. Reason: 'TypeError("cannot pickle '_thread.lock' object")'

In [ ]:
t_df = pd.read_csv('corrected_wildberries_reviews.csv.gz', compression='gzip')
t_df

,Unnamed: 0,review_full_text,review_rating,product,category,url,corrected_texts
0,0,Работает хорошо.,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Работает хорошо.
1,1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и..."
2,2,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ..."
3,3,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Лебёдка хорошая. Но в инструкции ни слова про ...
4,4,"Всё в комплекте, есть инструкция на русском яз...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Всё в комплекте, есть инструкция на русском яз..."
...,...,...,...,...,...,...,...
995,998,"Спасибо, вполне рабочая вещь",5,ПолиКомПласт / Преобразователь очиститель ржав...,/Автотовары/Автокосметика и автохимия,https://www.wildberries.ru/catalog/155565431/f...,"Спасибо, вполне рабочая вещь"
996,999,"так не о чем, эффекта почти нет, бестолковая х...",3,ПолиКомПласт / Преобразователь очиститель ржав...,/Автотовары/Автокосметика и автохимия,https://www.wildberries.ru/catalog/155565431/f...,"Так не о чем, эффекта почти нет, бестолковая х..."
997,1000,Посмотрим что и как будет,5,ПолиКомПласт / Преобразователь очиститель ржав...,/Автотовары/Автокосметика и автохимия,https://www.wildberries.ru/catalog/155565431/f...,Посмотрим что и как будет
998,1001,Отлично справляеться,5,ПолиКомПласт / Преобразователь очиститель ржав...,/Автотовары/Автокосметика и автохимия,https://www.wildberries.ru/catalog/155565431/f...,Отлично справляется


In [ ]:
t_df[(t_df["review_full_text"] != t_df["corrected_text"])]

KeyError: 'corrected_text'